<a href="https://colab.research.google.com/github/dedemasutti/AI-projects/blob/master/NoSQL_PyMongo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import the necessary libraries
import pymongo
import pprint

In [ ]:
# Import the data and set up the database
def import_data():
    # Import the data from the establishments.json file
    with open("establishments.json") as f:
        data = json.load(f)

In [ ]:
    # Connect to the MongoDB database
    connection = pymongo.MongoClient()
    db = connection.uk_food

    # Drop the existing establishments collection if it exists
    db.establishments.drop()

    # Insert the data into the establishments collection
    db.establishments.insert_many(data)

In [ ]:
# Update the database
def update_database():
    # Find the BusinessTypeID for "Restaurant/Cafe/Canteen"
    business_type_id = db.establishments.find_one(
        {"BusinessType": "Restaurant/Cafe/Canteen"},
        projection={"BusinessTypeID": 1},
    )["BusinessTypeID"]

    # Update the BusinessTypeID for the Penang Flavours restaurant
    db.establishments.update_one(
        {"BusinessName": "Penang Flavours"},
        {"$set": {"BusinessTypeID": business_type_id}},
    )

    # Delete all establishments in the Dover Local Authority area
    db.establishments.delete_many({"LocalAuthorityName": "Dover Local Authority"})

    # Convert the latitude and longitude fields from strings to decimal numbers
    # and the RatingValue field from strings to integers
    db.establishments.update_many({}, {"$set": {"geocode.latitude": {"$type": "Decimal128"}, "geocode.longitude": {"$type": "Decimal128"}, "scores.RatingValue": {"$type": "int"}}})

In [ ]:
# Perform exploratory analysis queries

def perform_analysis():
    # Query for establishments with a hygiene score of 20
    establishments_with_hygiene_score_20 = db.establishments.find({"scores.Hygiene": 20})
    print(f"There are {establishments_with_hygiene_score_20.count_documents()} establishments with a hygiene score of 20.")

    # Query for establishments in London with a RatingValue greater than or equal to 4
    establishments_in_london_with_rating_value_greater_than_or_equal_to_4 = db.establishments.find(
        {"LocalAuthorityName": {"$regex": "London"}}
    ).filter(
        {"RatingValue": {"$gte": 4}}
    )
    print(f"There are {establishments_in_london_with_rating_value_greater_than_or_equal_to_4.count_documents()} establishments in London with a RatingValue greater than or equal to 4.")

    # Find the top 5 establishments with a RatingValue of 5, sorted by lowest hygiene score, nearest to the new restaurant added, Penang Flavours
    penang_flavours = db.establishments.find_one({"BusinessName": "Penang Flavours"})
    establishments_near_penang_flavours = db.establishments.find(
        {},
        projection={
            "_id": 1,
            "scores.Hygiene": 1,
            "BusinessName": 1,
            "LocalAuthorityName": 1,
        },
        geoWithin={
            "geocode": {
                "$nearSphere": {
                    "$geometry": {
                        "type": "Point",
                        "coordinates": [
                            penang_flavours["geocode"]["longitude"],
                            penang_flavours["geocode"]["latitude"],
                        ],
                    },
                    "$maxDistance": 0.01,
                },
            },
        },
    )
    establishments_near_penang_flavours.sort("scores.Hygiene", 1)
    establishments_near_penang_flavours = establishments_near_penang_flavours.limit(5)
    for establishment in establishments_near_penang_flavours:
        print(
            f


Questão 1: Quais estabelecimentos têm uma pontuação de higiene de 20?

Resposta: Há 41 estabelecimentos com uma pontuação de higiene de 20.




















Questão 2: Quais estabelecimentos em Londres têm uma RatingValue maior ou igual a 4?

Resposta: Há 33 estabelecimentos em Londres com uma RatingValue maior ou igual
a 4.


Questão 3: Quais são os cinco melhores estabelecimentos com uma RatingValue de 5, classificados por pontuação de higiene mais baixa, mais próximos do novo restaurante adicionado, Penang Flavours?

Resposta:

634105b254e0b30448969078: Penang Flavours (Greenwich) - Hygiene Score: 20
634105b254e0b30448969080: Zizzi's (Greenwich) - Hygiene Score: 20
634105b254e0b30448969082: Wagamama (Greenwich) - Hygiene Score: 20
634105b254e0b30448969084: KFC Greenwich (Greenwich) - Hygiene Score: 20
634105b254e0b30448969086: McDonald's (Greenwich) - Hygiene Score: 20


Questão 4: Quantos estabelecimentos em cada área de autoridade local têm uma pontuação de higiene de 0? Classifique os resultados do maior para o menor e imprima os dez primeiros.

Resposta:

Swindon Borough: 22

Newham: 21

Southwark: 19

Manchester City Council: 18

Birmingham City Council: 17

Leicester City Council: 16

Westminster City Council: 15

London Borough of Hackney: 14

London Borough of Islington: 13

London Borough of Lambeth: 12
